In [1]:
from z3 import *
import numpy as np

Read instance file:

In [2]:
input_filename = '../../Instances/8x8.txt'

w, h, n, DX, DY = None, None, None, None, None

with open(input_filename, 'r') as f_in:
    lines = f_in.read().splitlines()

    split = lines[0].split(' ')
    w = int(split[0])
    h = int(split[1])

    n = int(lines[1])

    DX = []
    DY = []

    for i in range(int(n)):
        split = lines[i+2].split(' ')
        DX.append(int(split[0]))
        DY.append(int(split[1]))

Solver:

In [3]:
solver = Solver()

Model:

In [4]:
B = []

for i in range(h):
    B.append([])
    for j in range(w):
        B[i].append([])
        for k in range(n):
            B[i][j].append(Bool('B_{}_{}_{}'.format(i, j, k)))

In [5]:
B

[[[B_0_0_0, B_0_0_1, B_0_0_2, B_0_0_3],
  [B_0_1_0, B_0_1_1, B_0_1_2, B_0_1_3],
  [B_0_2_0, B_0_2_1, B_0_2_2, B_0_2_3],
  [B_0_3_0, B_0_3_1, B_0_3_2, B_0_3_3],
  [B_0_4_0, B_0_4_1, B_0_4_2, B_0_4_3],
  [B_0_5_0, B_0_5_1, B_0_5_2, B_0_5_3],
  [B_0_6_0, B_0_6_1, B_0_6_2, B_0_6_3],
  [B_0_7_0, B_0_7_1, B_0_7_2, B_0_7_3]],
 [[B_1_0_0, B_1_0_1, B_1_0_2, B_1_0_3],
  [B_1_1_0, B_1_1_1, B_1_1_2, B_1_1_3],
  [B_1_2_0, B_1_2_1, B_1_2_2, B_1_2_3],
  [B_1_3_0, B_1_3_1, B_1_3_2, B_1_3_3],
  [B_1_4_0, B_1_4_1, B_1_4_2, B_1_4_3],
  [B_1_5_0, B_1_5_1, B_1_5_2, B_1_5_3],
  [B_1_6_0, B_1_6_1, B_1_6_2, B_1_6_3],
  [B_1_7_0, B_1_7_1, B_1_7_2, B_1_7_3]],
 [[B_2_0_0, B_2_0_1, B_2_0_2, B_2_0_3],
  [B_2_1_0, B_2_1_1, B_2_1_2, B_2_1_3],
  [B_2_2_0, B_2_2_1, B_2_2_2, B_2_2_3],
  [B_2_3_0, B_2_3_1, B_2_3_2, B_2_3_3],
  [B_2_4_0, B_2_4_1, B_2_4_2, B_2_4_3],
  [B_2_5_0, B_2_5_1, B_2_5_2, B_2_5_3],
  [B_2_6_0, B_2_6_1, B_2_6_2, B_2_6_3],
  [B_2_7_0, B_2_7_1, B_2_7_2, B_2_7_3]],
 [[B_3_0_0, B_3_0_1, B_3_0_2, B_3_0_3

In [6]:
# constraint at most one color
for i in range(h):
    for j in range(w):
        for k1 in range(n):
            for k2 in range(k1+1, n):
                solver.add(Not(And([B[i][j][k1], B[i][j][k2]])))

In [7]:
all_cells = []
for p in range(n):
    dx = DX[p]
    dy = DY[p]
    
    present_cells = []
    
    for i in range(h-dy+1):
        for j in range(w-dx+1):
            # positions for the top left corner
            
            current_cells = []
            for f1 in range(dy):
                for f2 in range(dx):
                    # cells of the piece of paper
                    current_cells.append(B[i+f1][j+f2][p])                    
            
            present_cells.append(And(current_cells))
    
    #all_cells.append(Or(present_cells)) # XOR needed!
    for a in range(len(present_cells)):
        for b in range(a+1, len(present_cells)):
            all_cells.append(And(Not(And(present_cells[a], present_cells[b])), Or(present_cells)))
    
solver.add(And(all_cells))

In [8]:
solver.check()

sat

From Z3 model solution to file:

In [9]:
solution = np.zeros((h, w, n), dtype=bool)
model = solver.model()

for i in range(h):
    for j in range(w):
        for k in range(n):
            solution[i, j, k] = is_true(model[B[i][j][k]])

In [10]:
solution

array([[[False,  True, False, False],
        [False,  True, False, False],
        [False,  True, False, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False]],

       [[False,  True, False, False],
        [False,  True, False, False],
        [False,  True, False, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False]],

       [[False,  True, False, False],
        [False,  True, False, False],
        [False,  True, False, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False],
        [False, False,  True, False]],

       [[False,  True, False, False],
        [False,  True, False, False],
      

In [14]:
xy = {}
for p in range(n):
    # DEBUG
    print(solution[:, :, p])
    # DEBUG
    for i in range(h-1, -1, -1):
        for j in range(w):
            if solution[i, j, p] and p not in xy:
                xy[p] = [j, h-1-i]
                print('Color {}: [{}, {}]'.format(p, j, h-1-i))

[[False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [ True  True  True False False False False False]
 [ True  True  True False False False False False]
 [ True  True  True False False False False False]]
Color 0: [0, 0]
[[ True  True  True False False False False False]
 [ True  True  True False False False False False]
 [ True  True  True False False False False False]
 [ True  True  True False False False False False]
 [ True  True  True False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]
 [False False False False False False False False]]
Color 1: [0, 3]
[[False False False  True  True  True  True  True]
 [False False False  True  True  True  True  True]
 [False False False  True  True  True  True  Tru

In [12]:
xy

{0: [0, 0], 1: [0, 3], 2: [3, 5], 3: [3, 0]}

In [13]:
output_filename = '../../Instances/8x8_sol.txt'
with open(output_filename, 'w') as f_out:
    f_out.write('{} {}\n'.format(w, h))
    f_out.write('{}\n'.format(n))
    for i in range(n):
        f_out.write('{} {}\t{} {}\n'.format(DX[i], DY[i], xy[i][0], xy[i][1]))